### Basic Analysis on the CAGE data

In [143]:
import pandas as pd
import pyBigWig
from matplotlib import pyplot as plt
import seaborn as sns
import math
import numpy as np
from tqdm import tqdm
import os

In [144]:
DATA_PATH = "../data/ML4G_Project_1_Data"
CAGE_PATH = "CAGE-train/CAGE-train"
DNASE_PATH = "DNase-bed"

In [145]:
# Loading the data from the _info files
# there is X1, train and val (info and y)

dfs = {}
for i in range(3):
    for split in ["train", "val"] if i < 2 else ["test"]:
        info_path = os.path.join(DATA_PATH, CAGE_PATH, f"X{i+1}_{split}_info.tsv")
        df_info = pd.read_csv(info_path, sep="\t")
        dfs[f"X{i+1}_{split}_info"] = df_info

        if i == 2:
            continue
        y_path = os.path.join(DATA_PATH, CAGE_PATH, f"X{i+1}_{split}_y.tsv")
        df_y = pd.read_csv(y_path, sep="\t")
        dfs[f"X{i+1}_{split}_y"] = df_y

In [ ]:
dfs["X1_train_info"].head()

In [147]:
# assert that the lengths of the corresponding info and y files are the same
for i in range(2):
    for split in ["train", "val"]:
        assert len(dfs[f"X{i+1}_{split}_info"]) == len(dfs[f"X{i+1}_{split}_y"])

In [148]:
# Merge the info and y files, by adding the "gex" column from "y" to "info"
for i in range(2):
    for split in ["train", "val"]:
        df_info = dfs[f"X{i+1}_{split}_info"]
        df_y = dfs[f"X{i+1}_{split}_y"]
        df_info["gex"] = df_y["gex"]
        dfs[f"X{i+1}_{split}"] = df_info
        del dfs[f"X{i+1}_{split}_info"]
        del dfs[f"X{i+1}_{split}_y"]

In [ ]:
dfs.keys()

In [150]:
# Generating gene length and TSS length in "info" dataframes
for key in dfs.keys():
    df = dfs[key]
    df["gene_length"] = df["gene_end"] - df["gene_start"]
    df["tss_length"] = df["TSS_end"] - df["TSS_start"]

In [ ]:
# Display basic info and statistics
print("Data Summary:")
dfs["X1_train"].info()

In [ ]:
dfs["X1_val"].describe()

In [ ]:
# Min, avg, max gene expression (gex) values over all "y" dataframes
for key in dfs.keys():
    if key == "X3_test_info":
        continue
    df = dfs[key]
    print(f"{key}: Min: {df['gex'].min()}, Avg: {df['gex'].mean()}, Max: {df['gex'].max()}")

In [ ]:
print("\nSummary Statistics:")
dfs["X1_train"].describe()

In [ ]:
# Print, avg, min and max gene and TSS length for all "info" dataframes
for key in dfs.keys():
    df = dfs[key]
    print(f"\n{key}:")
    print(f"Gene length: min={df['gene_length'].min()}, avg={df['gene_length'].mean()}, max={df['gene_length'].max()}")
    print(f"TSS length: min={df['tss_length'].min()}, avg={df['tss_length'].mean()}, max={df['tss_length'].max()}")

In [ ]:
# Strand distribution
def plot_strand_distribution(df, title):
    # Define a color palette mapping + to one color and - to another
    strand_palette = {'+': 'skyblue', '-': 'salmon'}

    # Strand distribution with consistent order and colors
    plt.figure(figsize=(6, 4))
    sns.countplot(x='strand', data=df, hue='strand', palette=strand_palette, order=['+', '-'], legend=False)
    plt.title('Strand Distribution (+/-) for ' + title)
    plt.xlabel('Strand')
    plt.ylabel('Count')
    plt.show()


# Plot strand distribution for all "info" dataframes
for key in dfs.keys():
    plot_strand_distribution(dfs[key], key)

In [ ]:
# Gene expression distribution
def plot_gene_expression_distribution(df, title):
    plt.figure(figsize=(6, 4))
    sns.histplot(df["log_gex"], kde=True, color='skyblue')
    plt.title('Gene Expression Distribution for ' + title)
    plt.xlabel('log(gex + 1)')
    plt.ylabel('Density')
    plt.show()


# Iterate over all "y" dataframes and plot gene expression distribution 
# Apply first a log-transformation to the gene expression values
for key in dfs.keys():
    if key == "X3_test_info":
        continue
    dfs[key]["log_gex"] = dfs[key]["gex"].apply(lambda x: math.log(x + 1)) 
    plot_gene_expression_distribution(dfs[key], key) 

In [ ]:
# min, avg, max of log-transformed gene expression values
for key in dfs.keys():
    if key == "X3_test_info":
        continue
    df = dfs[key]
    print(f"{key} - Log-GEX: Min: {df['log_gex'].min()}, Avg: {df['log_gex'].mean()}, Max: {df['log_gex'].max()}")
    print(f"{key} - GEX: Min: {df['gex'].min()}, Avg: {df['gex'].mean()}, Max: {df['gex'].max()}")
    print()

In [ ]:
# Count how many records have a gene expression value lower than 0
for key in dfs.keys():
    if key == "X3_test_info":
        continue
    df = dfs[key]
    print(f"{key}: {df[df['gex'] < 0].shape[0]} records have a gene expression value lower than 1")

### Histone Marks Data Analysis

In [176]:
HISTONE_MARKS = ["H3K4me1", "H3K4me3", "H3K9me3", "H3K27ac", "H3K36me3", "H3K27me3"]

region_map = {
    "H3K4me1": "enhancer",
    "H3K4me3": "promoter",
    "H3K9me3": "heterochromatin",
    "H3K27ac": "enhancer/promoter",
    "H3K36me3": "gene body",
    "H3K27me3": "promoter"
}

effect_map = {
    "H3K4me1": "poised/active enhancers",
    "H3K4me3": "active",
    "H3K27ac": "active",
    "H3K36me3": "transcriptional elongation",
    "H3K9me3": "repressive",
    "H3K27me3": "repressive"
}

# Activating Marks:

# H3K4me3: Active transcription (promoters).
# H3K27ac: Active transcription (enhancers/promoters).
# H3K4me1: Poised/active enhancers (enhancers, sometimes promoters).
# H3K36me3: Associated with transcriptional elongation (gene bodies).

# Repressive Marks:

# H3K9me3: Repressive mark associated with heterochromatin (gene silencing).
# H3K27me3: Repressive mark linked to Polycomb-mediated gene silencing (reversible gene silencing).

In [162]:
bigwig_dfs = {}
# we have under each DATA_PATH/HISTONE_MARK-bigwig folder X1, X2 and X2 .bw / .bigwig files
for mark in HISTONE_MARKS:
    bigwig_dfs[mark] = {}
    for i in range(3):
        bigwig_path = os.path.join(DATA_PATH, f"{mark}-bigwig", f"X{i+1}.bigwig")

        # check if it exists, otherwise its .bw
        if not os.path.exists(bigwig_path):
            bigwig_path = os.path.join(DATA_PATH, f"{mark}-bigwig", f"X{i+1}.bw")

        # Open bigwig file
        bw = pyBigWig.open(bigwig_path)
        bigwig_dfs[mark][f"X{i+1}"] = bw

In [ ]:
bigwig_dfs["H3K4me1"]["X1"].header()

In [ ]:
def extract_bigwig_signal(df, bw, region_size=1000):
    signals = []
    with tqdm(total=len(df)) as pbar:
        for idx, row in df.iterrows():
            pbar.update(1)

            # Get signal around TSS (+/- region_size bp)
            start = max(0, row['TSS_start'] - region_size)
            end = row['TSS_end'] + region_size
            signal = bw.values(row['chr'], start, end)
            signals.append(sum(signal) / len(signal))  # Example: mean signal
    return signals

# Extract signal for all histone marks and all "info" dataframes
for mark in HISTONE_MARKS:
    for key in dfs.keys():
        print(f"Extracting signal for {mark} - {key}")
        df = dfs[key]
        bw = bigwig_dfs[mark][key.split("_")[0]]
        df[mark] = extract_bigwig_signal(df, bw)

In [ ]:
dfs["X1_train"].head()

In [ ]:
# Plot histone signal vs. gene expression
def plot_histone_signal_vs_gene_expression(df, mark, split):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=mark, y='log_gex', data=df)
    plt.title(f'{mark} Signal ({region_map[mark]}, {effect_map[mark]}) vs. Gene Expression for {split}')
    plt.xlabel(f'Mean {mark} Signal Around TSS')
    plt.ylabel('Gene Expression (CPM)')
    plt.show()

# Iterate over all histone marks and plot signal vs. gene expression
for mark in HISTONE_MARKS:
    # plot_histone_signal_vs_gene_expression(dfs["X1_train"], mark, "X1_train")
    for key in dfs.keys():
        if key == "X3_test_info":
            continue
        plot_histone_signal_vs_gene_expression(dfs[key], mark, key)